In [45]:
import numpy as np 
import pandas as pd 
import random
import matplotlib.pyplot as plt 
import os
pd.set_option('display.max_rows', 200)
%matplotlib inline

# Import Data 

In [46]:
# define folder where data resides 
DATAFOLDER = "~/Documents/data-science-coursework/nyu-ml/project/data/"

### Appeals

In [47]:
# main table 
tblAppeal = pd.read_csv(os.path.join(DATAFOLDER, 'raw/tblAppeal.csv'), low_memory=False) 
print(tblAppeal.info())
tblAppeal.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870388 entries, 0 to 870387
Data columns (total 17 columns):
idnAppeal             870388 non-null int64
idncase               868758 non-null float64
idnProceeding         776380 non-null float64
strAppealCategory     870388 non-null object
strAppealType         870388 non-null object
datAppealFiled        870226 non-null object
strFiledBy            870378 non-null object
datAttorneyE27        521008 non-null object
datBIADecision        847196 non-null object
strBIADecision        847180 non-null object
strBIADecisionType    822882 non-null object
strCaseType           824855 non-null object
strLang               773811 non-null object
strNat                777004 non-null object
strProceedingIHP      367364 non-null object
strCustody            666356 non-null object
strProbono            1880 non-null object
dtypes: float64(2), int64(1), object(14)
memory usage: 112.9+ MB
None


,idnAppeal,idncase,idnProceeding,strAppealCategory,strAppealType,datAppealFiled,strFiledBy,datAttorneyE27,datBIADecision,strBIADecision,strBIADecisionType,strCaseType,strLang,strNat,strProceedingIHP,strCustody,strProbono
57910,57973,2518519.0,709595.0,IJ,Case Appeal,1995-07-31 00:00:00,A,NaN,1996-04-18 00:00:00,D30,P,DEP,SP,ES,NaN,R,NaN
759767,4934004,6684431.0,NaN,DD,DD Visa,2010-06-17 00:00:00,A,NaN,2011-06-30 00:00:00,REM,R,DDC,NaN,NaN,NaN,NaN,NaN
543838,4715250,4456728.0,3036352.0,IJ,Case Appeal,2005-11-17 00:00:00,I,2005-12-01 00:00:00,2007-07-23 00:00:00,REM,R,RMV,SP,MX,NaN,N,NaN
253608,4411373,4321089.0,2867156.0,IJ,Case Appeal,1999-03-02 00:00:00,A,1999-03-02 00:00:00,1999-07-12 00:00:00,DIS,P,RMV,SP,MX,,D,NaN
327233,4489366,3895350.0,2295633.0,IJ,Case Appeal,2001-04-23 00:00:00,A,NaN,2002-05-22 00:00:00,DIS,P,RMV,CRE,HA,,N,NaN


In [48]:
# drop appeals with no case number, proceeding number, or decision 
tblAppeal.dropna(subset=['idncase', 'idnProceeding', 'strBIADecision'], inplace=True) 
tblAppeal = tblAppeal[tblAppeal['idnProceeding'] != 0] # drop zeros 

# convert indexes to integers 
tblAppeal['idncase'] = tblAppeal['idncase'].astype(int) 
tblAppeal['idnProceeding'] = tblAppeal['idnProceeding'].astype(int) 
tblAppeal.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755222 entries, 0 to 869457
Data columns (total 17 columns):
idnAppeal             755222 non-null int64
idncase               755222 non-null int64
idnProceeding         755222 non-null int64
strAppealCategory     755222 non-null object
strAppealType         755222 non-null object
datAppealFiled        755060 non-null object
strFiledBy            755212 non-null object
datAttorneyE27        478717 non-null object
datBIADecision        755222 non-null object
strBIADecision        755222 non-null object
strBIADecisionType    731103 non-null object
strCaseType           755180 non-null object
strLang               752329 non-null object
strNat                754061 non-null object
strProceedingIHP      365943 non-null object
strCustody            643730 non-null object
strProbono            1842 non-null object
dtypes: int64(3), object(14)
memory usage: 103.7+ MB


In [49]:
# format column types 
tblAppeal['datAppealFiled_dt'] = pd.to_datetime(tblAppeal['datAppealFiled'], errors='coerce')
tblAppeal['datBIADecision_dt'] = pd.to_datetime(tblAppeal['datBIADecision'], errors='coerce')
tblAppeal['datAppealFiled_year'] = tblAppeal['datAppealFiled_dt'].dt.year

In [50]:
# code lookup tables 
bia_appeal_category = pd.read_excel(os.path.join(DATAFOLDER, 'raw/BIA Appeal Data File code translations.xlsx'), 
                                    sheetname='BIA Appeal Category', skip_footer=7)
bia_appeal_type = pd.read_excel(os.path.join(DATAFOLDER, 'raw/BIA Appeal Data File code translations.xlsx'),
                                sheetname='BIA Appeal Type', skip_footer=3)
bia_decision_type = pd.read_excel(os.path.join(DATAFOLDER, 'raw/BIA Appeal Data File code translations.xlsx'),
                                  sheetname='BIA decision type', skip_footer=2)
bia_decision_code = pd.read_excel(os.path.join(DATAFOLDER, 'raw/BIA Appeal Data File code translations.xlsx'),
                                  sheetname='BIA decision code', skip_footer=2)

In [51]:
# join them 
tblAppeal_df = tblAppeal.merge(bia_appeal_category, how='left', left_on='strAppealCategory', right_on='Code')\
                        .rename(columns={'Description': 'strAppealCategoryDesc'}).drop('Code', axis=1)\
                        .merge(bia_appeal_type, how='left', left_on='strAppealType', right_on='Code')\
                        .rename(columns={'Description': 'strAppealTypeDesc'}).drop('Code', axis=1)\
                        .merge(bia_decision_code, how='left', left_on='strBIADecision', right_on='Code')\
                        .rename(columns={'Description': 'strBIADecisionDesc'}).drop('Code', axis=1)\
                        .merge(bia_decision_type, how='left', left_on='strBIADecisionType', right_on='Code')\
                        .rename(columns={'Description': 'strBIADecisionTypeDesc'}).drop('Code', axis=1)
print(tblAppeal_df.info()) 
tblAppeal_df.sample(10).T

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755222 entries, 0 to 755221
Data columns (total 24 columns):
idnAppeal                 755222 non-null int64
idncase                   755222 non-null int64
idnProceeding             755222 non-null int64
strAppealCategory         755222 non-null object
strAppealType             755222 non-null object
datAppealFiled            755060 non-null object
strFiledBy                755212 non-null object
datAttorneyE27            478717 non-null object
datBIADecision            755222 non-null object
strBIADecision            755222 non-null object
strBIADecisionType        731103 non-null object
strCaseType               755180 non-null object
strLang                   752329 non-null object
strNat                    754061 non-null object
strProceedingIHP          365943 non-null object
strCustody                643730 non-null object
strProbono                1842 non-null object
datAppealFiled_dt         755060 non-null datetime64[ns]
datB

,90076,631767,27278,398447,120500,356514,341245,220336,197409,340965
idnAppeal,90220,4867654,27327,4604702,141077,4559587,4543188,4403692,4377949,4542895
idncase,2703095,3381175,2325657,3131862,3477255,2992575,4092204,3290960,3158413,4247254
idnProceeding,956528,1680960,468751,1495683,1795329,1327630,2566699,46614,1528086,2776191
strAppealCategory,IJ,IJ,IJ,IJ,IJ,IJ,IJ,IJ,IJ,IJ
strAppealType,Case Appeal,MTR BIA,Case Appeal,Case Appeal,Case Appeal,MTR BIA,Case Appeal,MTR BIA,Case Appeal,Appeal of IJ MTR
datAppealFiled,1984-12-28 00:00:00,2009-05-22 00:00:00,1988-04-22 00:00:00,2003-11-06 00:00:00,1996-05-23 00:00:00,2002-12-23 00:00:00,2002-09-05 00:00:00,1998-12-10 00:00:00,1998-03-19 00:00:00,2002-09-03 00:00:00
strFiledBy,A,I,A,A,A,A,A,A,A,A
datAttorneyE27,NaN,NaN,NaN,2003-11-06 00:00:00,1996-05-23 00:00:00,2002-12-23 00:00:00,2002-09-05 00:00:00,1998-12-10 00:00:00,NaN,2002-09-03 00:00:00
datBIADecision,1989-07-25 00:00:00,2009-06-29 00:00:00,1991-01-29 00:00:00,2005-03-07 00:00:00,1997-03-20 00:00:00,2003-02-13 00:00:00,2004-02-09 00:00:00,2000-12-04 00:00:00,2002-06-07 00:00:00,2003-09-16 00:00:00
strBIADecision,SUS,REM,D30,D30,D30,NJU,DIS,GRN,SAV,REM


In [52]:
# some strBIADecision don't have corresponding code translations; to remove 
print(tblAppeal_df[pd.isnull(tblAppeal_df['strBIADecisionDesc'])]['strBIADecision'].value_counts()) 
tblAppeal_df.dropna(subset=['strBIADecisionDesc'], inplace=True)
tblAppeal_df.info()

DSO    5987
DED    2107
CPG    1704
ABC    1606
APD    1170
GRS     431
DNS     421
ADM     355
DMO     238
CPC      90
MB       88
SNC      82
AFD      54
RET      52
ADD      18
CGR      16
TPD      16
REV      13
WPD       5
SUP       4
          2
Name: strBIADecision, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 740763 entries, 0 to 755221
Data columns (total 24 columns):
idnAppeal                 740763 non-null int64
idncase                   740763 non-null int64
idnProceeding             740763 non-null int64
strAppealCategory         740763 non-null object
strAppealType             740763 non-null object
datAppealFiled            740609 non-null object
strFiledBy                740753 non-null object
datAttorneyE27            470111 non-null object
datBIADecision            740763 non-null object
strBIADecision            740763 non-null object
strBIADecisionType        717284 non-null object
strCaseType               740721 non-null object
strLang          

### Master Proceedings

In [53]:
master = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master.csv')) 
print(master.info())
master.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6084437 entries, 0 to 6084436
Data columns (total 16 columns):
idncase             float64
nat                 object
case_type           object
c_asy_type          object
idnproceeding       object
base_city_code      object
hearing_loc_code    object
dec_type            object
dec_code            object
other_comp          object
osc_date            object
input_date          object
comp_date           object
attorney_flag       float64
ij_code             object
tracid              float64
dtypes: float64(3), object(13)
memory usage: 742.7+ MB
None


,idncase,nat,case_type,c_asy_type,idnproceeding,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid
5664535,6888070.0,MX,RMV,I,5698138,ATL,ATL,NaN,NaN,NaN,16NOV2011,21NOV2011,NaN,1.0,EBW,132.0
732053,2617730.0,NU,DEP,E,836416,SFR,SFR,NaN,NaN,A,03APR1989,27OCT1998,28MAY1999,1.0,MJY,207.0
4748833,6142708.0,MX,RMV,NaN,4725980,ADL,LAN,O,X,NaN,29DEC2008,07JAN2009,18FEB2009,1.0,ROV,79.0
1629692,3305070.0,HO,DEP,NaN,28543,NYC,NYC,O,D,NaN,13JUL1994,06SEP1994,26OCT1994,NaN,WAC,4.0
1541146,3233838.0,CU,DEP,NaN,319774,PIS,PIS,NaN,NaN,T,06SEP1993,15SEP1993,21SEP1993,NaN,JZ,50.0


In [54]:
# check missing data 
master.isnull().sum()

idncase                  14
nat                   16775
case_type                15
c_asy_type          4449366
idnproceeding             0
base_city_code           24
hearing_loc_code       1516
dec_type            1519549
dec_code            1674748
other_comp          4744870
osc_date              33353
input_date            29744
comp_date            335076
attorney_flag       3044591
ij_code               39598
tracid               341914
dtype: int64

In [67]:
# look up tables 
master_case_type = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master_case_type.csv'))
master_decision_type = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master_decision_type.csv'))
# master_decision_on_proceeding = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master_decision_on_proceeding.csv'), 
#                                              skipfooter=2, engine='python')
master_decision_on_proceeding = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master_decision_on_proceeding.csv'))
master_decision_on_proceeding.drop(master_decision_on_proceeding.tail(2).index, inplace=True)
master_decision_on_proceeding.drop(['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)

In [68]:
# add column to indicate whether decision was listed in dec_code or other_comp 
master['dec_col'] = np.where(pd.notnull(master['dec_code']), 'C', 'O') 
master['dec_judge'] = np.where(pd.notnull(master['dec_code']), master['dec_code'], master['other_comp']) 

In [69]:
# join to look up tables
master_df = master.merge(master_decision_type, how='left', left_on='dec_type', right_on='strCode')\
                  .rename(columns={'strDescription': 'dec_type_str'}).drop('strCode', axis=1)\
                  .merge(master_case_type, how='left', left_on='case_type', right_on='strCode')\
                  .rename(columns={'strDescription': 'case_type_str'}).drop('strCode', axis=1)\
                  .merge(master_decision_on_proceeding, how='left', left_on=['case_type', 'dec_judge', 'dec_col'], 
                         right_on=['strCaseType', 'strDecCode', 'strDecType'])\
                  .rename(columns={'strDecDescription': 'dec_code_str'}).drop(['strCaseType', 'strDecCode', 'strDecType'], axis=1)
master_df.sample(5)

,idncase,nat,case_type,c_asy_type,idnproceeding,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,...,input_date,comp_date,attorney_flag,ij_code,tracid,dec_col,dec_judge,dec_type_str,case_type_str,dec_code_str
4611552,6027386.0,ES,RMV,NaN,4580316,BAL,BAL,W,V,NaN,...,30JUN2008,10DEC2008,NaN,JFG,10.0,C,V,written decision,Removal,Voluntary Departure
4749994,6143669.0,MX,RMV,NaN,4727235,CHI,CHD,NaN,NaN,T,...,12JAN2009,13JAN2009,1.0,GPK,30.0,O,T,NaN,Removal,Transfer
3628810,5190006.0,ES,RMV,NaN,3533275,HLG,HLG,W,X,NaN,...,19MAY2005,08JUL2005,NaN,WCP,62.0,C,X,written decision,Removal,Remove
1230697,2987277.0,BG,RMV,I,1320863,NYC,NYC,NaN,NaN,C,...,12MAY1997,05NOV1997,1.0,WFJ,276.0,O,C,NaN,Removal,Change of Venue
5325265,6619733.0,MX,RMV,NaN,5330187,OAK,JNA,NaN,NaN,T,...,15OCT2010,04NOV2010,NaN,JAD,166.0,O,T,NaN,Removal,Transfer


### Master Proceedings (processed data from Daniel Chen)

In [85]:
master_dchen = pd.read_stata(os.path.join(DATAFOLDER,'decision_sched_merge_adjdate.dta')) 

In [86]:
master_dchen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5664753 entries, 0 to 5664752
Columns: 106 entries, idncase to flag_datemismatch
dtypes: category(12), float32(2), float64(50), int16(3), int32(3), int8(4), object(32)
memory usage: 3.7+ GB


In [ ]:
# import sys

# reader = pd.read_stata(os.path.join(DATAFOLDER,'decision_sched_merge_adjdate.dta'), iterator=True)
# df = pd.DataFrame()
# chunk_size = 100*1000
# try:
#     chunk = reader.get_chunk(chunk_size)
#     with open(os.path.join(DATAFOLDER,'decision_sched_merge_adjdate.csv'), 'a') as f:
#         first = True
#         while len(chunk) > 0:
#             if first:
#                 chunk.to_csv(f, header=True)
#                 first = False
#             else:
#                 chunk.to_csv(f, header=False)
# #             df = df.append(chunk, ignore_index=True)
#             chunk = reader.get_chunk(chunk_size)
#             print('.')
#             sys.stdout.flush()
# except (StopIteration, KeyboardInterrupt):
#     pass

# master_dchen.info()

In [ ]:
# master_dchen = pd.read_csv(os.path.join(DATAFOLDER,'decision_sched_merge_adjdate.csv'), 
#                            low_memory=False) 
# master_dchen.info()

In [87]:
# check for nulls 
master_dchen.isnull().sum()

idncase                                 0
nat                                     0
case_type                               0
c_asy_type                              0
idnproceeding                           0
base_city_code                          0
hearing_loc_code                        0
dec_type                                0
dec_code                                0
other_comp                              0
attorney_flag                     2907462
ij_code                                 0
tracid                             248615
comp_year                               0
comp_month                              0
comp_day                                0
comp_date                               0
osc_year                            32420
osc_month                           32420
osc_day                             32420
osc_date                            32420
input_year                          27549
input_month                         27549
input_day                         

In [88]:
master_dchen.sample(3).T

,5350032,727086,3550788
idncase,6180854,2710176,5515466
nat,CH,RP,ES
case_type,RMV,RMV,RMV
c_asy_type,E,E,
idnproceeding,4805909,965694,3934408
base_city_code,PHO,SFR,LOS
hearing_loc_code,PHO,SFD,LOS
dec_type,O,,O
dec_code,,,X
other_comp,C,T,


In [89]:
# add column to indicate whether decision was listed in dec_code or other_comp 
master_dchen['dec_col'] = np.where(pd.notnull(master_dchen['dec_code']), 'C', 'O') 
master_dchen['dec_judge'] = np.where(pd.notnull(master_dchen['dec_code']), master_dchen['dec_code'], master_dchen['other_comp']) 

In [90]:
# join to look up tables
master_dchen_df = master_dchen.merge(master_decision_type, how='left', left_on='dec_type', right_on='strCode')\
                              .rename(columns={'strDescription': 'dec_type_str'}).drop('strCode', axis=1)\
                              .merge(master_case_type, how='left', left_on='case_type', right_on='strCode')\
                              .rename(columns={'strDescription': 'case_type_str'}).drop('strCode', axis=1)\
                              .merge(master_decision_on_proceeding, how='left', 
                                     left_on=['case_type', 'dec_judge', 'dec_col'], 
                                     right_on=['strCaseType', 'strDecCode', 'strDecType'])\
                              .rename(columns={'strDecDescription': 'dec_code_str'}).drop(['strCaseType', 'strDecCode', 'strDecType'], axis=1)
master_dchen_df.sample(3).T

,4831740,2060320,2286404
idncase,2381337,3870856,4053929
nat,NU,BL,CH
case_type,DEP,RMV,RMV
c_asy_type,E,,I
idnproceeding,539199,2265248,2509545
base_city_code,PIS,LOS,LOS
hearing_loc_code,PIS,LOS,LOS
dec_type,,O,O
dec_code,,R,R
other_comp,C,,


### Master Proceedings (processed by Sagent/Dunn)

In [91]:
master_dunn = pd.read_csv(os.path.join(DATAFOLDER, 
                                       'AsylumAdj/data_for_model/_decision_scheduling_merge_final_converted.csv'), 
                          encoding='latin-1', low_memory=False) # gets UnicodeDecodeError otherwise 
master_dunn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602500 entries, 0 to 602499
Columns: 182 entries, Unnamed: 0 to last_hearing_on_comp_date
dtypes: bool(1), float64(72), int64(39), object(70)
memory usage: 832.6+ MB


In [92]:
# dunn has far fewer proceedings than the original 
print "Original master has {} data points, Dunn's has {}".format(len(master_dunn), len(master))

Original master has 602500 data points, Dunn's has 6084437


In [93]:
print master_dchen.columns.tolist()

['idncase', 'nat', 'case_type', 'c_asy_type', 'idnproceeding', 'base_city_code', 'hearing_loc_code', 'dec_type', 'dec_code', 'other_comp', 'attorney_flag', 'ij_code', 'tracid', 'comp_year', 'comp_month', 'comp_day', 'comp_date', 'osc_year', 'osc_month', 'osc_day', 'osc_date', 'input_year', 'input_month', 'input_day', 'input_date', 'lawyer', 'defensive', 'affirmative', 'case_type_string', '_mcase', 'dec_type_string', '_mdectype', 'outcome_recorded_in_field', 'dec_string', '_mdecproceeddec', '_mdecproceedoth', 'nat_string', '_mnat', 'base_city_street', 'base_city_string', 'base_city_state', 'base_city_zip5', 'base_city_zip4', 'base_city_phone', '_mbasecity', 'hearing_loc_string1', 'hearing_loc_string2', 'hearing_loc_street', 'hearing_loc_city', 'hearing_loc_state', 'hearing_loc_zip5', 'hearing_loc_phone', '_mhearingloc', 'judge_name_caps', '_mlookupijcode', 'cityid', 'judgeid', 'natid', 'venue_change', 'deport', 'relief_granted', 'remove', 'terminated', 'voluntary_departure', 'oral', 'wr

In [94]:
master_dchen['asylum_only_proceeding'].value_counts()

0.0    5645688
1.0      18953
Name: asylum_only_proceeding, dtype: int64

In [95]:
master_dunn['case_type_string'].value_counts()

REMOVAL                 408738
DEPORTATION             142533
EXCLUSION                39062
ASYLUM ONLY CASE          9669
WITHHOLDING ONLY          2487
CREDIBLE FEAR REVIEW         1
Name: case_type_string, dtype: int64

In [96]:
master_dunn['asylum'].value_counts()

1    562348
0     40152
Name: asylum, dtype: int64

In [103]:
# determine what % of master proceedings that Dunn retained 
master_dunn_pairs = master_dunn[['idncase', 'idnproceeding', 'grant']].copy()
master_dunn_pairs['in_dunn'] = 1 
master_dunn_delta = master_df[['idncase', 'idnproceeding', 'case_type_str']].merge(master_dunn_pairs, how='left', 
                                                                                   on=['idncase', 'idnproceeding']) 
master_dunn_delta['in_dunn'].fillna(0, inplace=True)
master_dunn_delta.groupby(['case_type_str'])['in_dunn'].agg(['size', np.mean])

,size,mean
case_type_str,,
Asylum Only Case,19881,0.447261
Claimed Status Review,1262,0.000000
Continued Detention Review,42,0.000000
Credible Fear Review,7453,0.000134
Deportation,1429597,0.099371
Exclusion,258052,0.151125
NACARA Adjustment,470,0.000000
Reasonable Fear Case,3611,0.000000
Recission,2087,0.000000


# Determine scope of appeal case types

In [104]:
appeals_in_scope = tblAppeal_df.merge(master_dunn_pairs, how='left', 
                                      left_on=['idncase', 'idnProceeding'], right_on=['idncase', 'idnproceeding']) 
appeals_in_scope = appeals_in_scope.merge(master_dchen[['idncase', 'idnproceeding', 'case_type_string']], 
                                          how='left', left_on=['idncase', 'idnProceeding'], 
                                          right_on=['idncase', 'idnproceeding']) 
appeals_in_scope['in_dunn'].fillna(0, inplace=True)
appeals_in_scope['case_type_string'].fillna('NOT_MATCHED_TO_MASTER', inplace=True)

In [105]:
# distribution of appeal vs. proceeding types - all
appeals_in_scope.groupby(['strAppealTypeDesc', 'case_type_string']).size().unstack().fillna(0) 

case_type_string,,ASYLUM ONLY CASE,CLAIMED STATUS REVIEW,CONTINUED DETENTION REVIEW,CREDIBLE FEAR REVIEW,DEPORTATION,EXCLUSION,NACARA ADJUSTMENT,NOT_MATCHED_TO_MASTER,REASONABLE FEAR CASE,RECISSION,REMOVAL,WITHHOLDING ONLY
strAppealTypeDesc,,,,,,,,,,,,,
Appeal of IJ MTR,3.0,168.0,1.0,0.0,0.0,16298.0,2058.0,4.0,358.0,0.0,6.0,21347.0,12.0
Bond Appeal,1.0,19.0,0.0,0.0,1.0,5511.0,14.0,0.0,1589.0,1.0,0.0,15259.0,6.0
Bond MTR Reconsider,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11.0,0.0,0.0,84.0,0.0
Bond MTR Reinstate,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,25.0,0.0
Bond MTR Reopen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,46.0,0.0
Case Appeal,10.0,4251.0,9.0,0.0,101.0,140490.0,27977.0,56.0,9157.0,87.0,380.0,322863.0,974.0
Circuit Court Remand,0.0,197.0,0.0,0.0,0.0,1170.0,208.0,0.0,55.0,0.0,0.0,11134.0,27.0
Interlocutory Appeal,1.0,20.0,0.0,0.0,0.0,633.0,297.0,0.0,328.0,2.0,5.0,2101.0,9.0
MTR BIA,2.0,1347.0,0.0,5.0,4.0,40877.0,7473.0,8.0,1427.0,5.0,24.0,104105.0,93.0


In [106]:
# distribution of appeal vs. proceeding types - what dunn included 
appeals_in_scope[appeals_in_scope['in_dunn'] == 1].groupby(['strAppealTypeDesc', 'case_type_string']).size().unstack().fillna(0) 

case_type_string,,ASYLUM ONLY CASE,DEPORTATION,EXCLUSION,REMOVAL,WITHHOLDING ONLY
strAppealTypeDesc,,,,,,
Appeal of IJ MTR,0.0,93.0,2653.0,478.0,3026.0,9.0
Bond Appeal,0.0,10.0,569.0,7.0,2388.0,6.0
Bond MTR Reconsider,0.0,0.0,0.0,0.0,12.0,0.0
Bond MTR Reinstate,0.0,0.0,0.0,0.0,4.0,0.0
Bond MTR Reopen,0.0,0.0,0.0,0.0,10.0,0.0
Case Appeal,2.0,3878.0,66912.0,18341.0,173865.0,842.0
Circuit Court Remand,0.0,186.0,604.0,145.0,7058.0,26.0
Interlocutory Appeal,1.0,7.0,68.0,41.0,486.0,7.0
MTR BIA,0.0,1231.0,21833.0,5739.0,59309.0,87.0


#### Final Criteria: appeal types in ['Appeal of IJ MTR', 'Case Appeal', 'Circuit Court Remand', 'Interlocutory Appeal', 'MTR BIA'] and is found in Dunn's 

In [101]:
# filter relevant appeal types 
selected_appeal_types = ['Appeal of IJ MTR', 'Case Appeal', 'Circuit Court Remand', 'Interlocutory Appeal', 'MTR BIA'] 
appeals_in_scope = appeals_in_scope[appeals_in_scope['strAppealTypeDesc'].isin(selected_appeal_types) & 
                                    appeals_in_scope['in_dunn'] == 1].copy() 
appeals_in_scope.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366927 entries, 3 to 740756
Data columns (total 28 columns):
idnAppeal                 366927 non-null int64
idncase                   366927 non-null int64
idnProceeding             366927 non-null int64
strAppealCategory         366927 non-null object
strAppealType             366927 non-null object
datAppealFiled            366863 non-null object
strFiledBy                366918 non-null object
datAttorneyE27            255225 non-null object
datBIADecision            366927 non-null object
strBIADecision            366927 non-null object
strBIADecisionType        356952 non-null object
strCaseType               366926 non-null object
strLang                   366119 non-null object
strNat                    366752 non-null object
strProceedingIHP          183004 non-null object
strCustody                309504 non-null object
strProbono                750 non-null object
datAppealFiled_dt         366863 non-null datetime64[ns]
datBI

#### Additional Criterion: Original Proceeding Outcome Must Not Be Granted (likely government appeal)

In [108]:
appeals_in_scope['grant'].value_counts()

0.0    357958
1.0     11975
Name: grant, dtype: int64

In [110]:
appeals_in_scope = appeals_in_scope[appeals_in_scope['grant'] == 0].copy() 
appeals_in_scope.drop('grant', axis=1, inplace=True)
appeals_in_scope.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357958 entries, 3 to 740756
Data columns (total 28 columns):
idnAppeal                 357958 non-null int64
idncase                   357958 non-null int64
idnProceeding             357958 non-null int64
strAppealCategory         357958 non-null object
strAppealType             357958 non-null object
datAppealFiled            357899 non-null object
strFiledBy                357949 non-null object
datAttorneyE27            248512 non-null object
datBIADecision            357958 non-null object
strBIADecision            357958 non-null object
strBIADecisionType        348620 non-null object
strCaseType               357953 non-null object
strLang                   357162 non-null object
strNat                    357775 non-null object
strProceedingIHP          179372 non-null object
strCustody                302173 non-null object
strProbono                625 non-null object
datAppealFiled_dt         357899 non-null datetime64[ns]
datBI

# Assign Labels 

In [111]:
# many different labels 
appeals_in_scope.groupby(['strBIADecisionDesc', 'strAppealTypeDesc']).size().unstack().fillna(0)

strAppealTypeDesc,Appeal of IJ MTR,Bond Appeal,Bond MTR Reconsider,Bond MTR Reinstate,Bond MTR Reopen,Case Appeal,Circuit Court Remand,Interlocutory Appeal,MTR BIA
strBIADecisionDesc,,,,,,,,,
Administrative Return,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,9.0
Background Check Remand,12.0,0.0,0.0,0.0,0.0,2840.0,532.0,0.0,93.0
Continued,2.0,0.0,0.0,0.0,0.0,1165.0,22.0,0.0,23.0
Denied,4.0,5.0,6.0,0.0,5.0,24.0,28.0,0.0,54855.0
Dismiss Appeal/Affirm IJ's Decision,3242.0,1172.0,0.0,0.0,0.0,95118.0,749.0,169.0,15.0
Dismiss as Untimely,101.0,32.0,0.0,0.0,0.0,1749.0,0.0,1.0,27.0
Dismissed (Grant V/D 30 days),3.0,0.0,0.0,0.0,0.0,30365.0,1.0,2.0,19.0
Dismissed (Voluntary Departure Granted),23.0,0.0,0.0,0.0,0.0,11233.0,198.0,0.0,0.0
Grant With No Remand,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,961.0


In [112]:
# assign positive and negative labels
granted_decisions = ['Background Check Remand', 'Grant With No Remand', 'Granted', 'Remand', 
                     'Sustain', 'Temporary Protected Status', 'Termination']
denied_decisions = ['Denied', "Dismiss Appeal/Affirm IJ's Decision", 'Dismissed (Grant V/D 30 days)', 
                    'Dismissed (Voluntary Departure Granted)', 'Rejection', 'SUMMARY AFFIRMANCE/VD', 
                    'Summary Affirmance', 'Summary Dismiss', 'Summary Dismissal (O) Other', 
                    'Summary Dismissal (a) inad reason on appeal', 'Summary Dismissal - Both (a) & (e)']
appeals_in_scope['granted'] = np.where(appeals_in_scope['strBIADecisionDesc'].isin(granted_decisions), 1, 
                                       np.where(appeals_in_scope['strBIADecisionDesc'].isin(denied_decisions), 0, None))

# drop neutral labels where appeals were dismissed due to administrative/procedural failures 
appeals_in_scope.dropna(subset=['granted'], inplace=True)
appeals_in_scope['granted'] = appeals_in_scope['granted'].astype(int)
appeals_in_scope.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335607 entries, 3 to 740756
Data columns (total 29 columns):
idnAppeal                 335607 non-null int64
idncase                   335607 non-null int64
idnProceeding             335607 non-null int64
strAppealCategory         335607 non-null object
strAppealType             335607 non-null object
datAppealFiled            335552 non-null object
strFiledBy                335598 non-null object
datAttorneyE27            236468 non-null object
datBIADecision            335607 non-null object
strBIADecision            335607 non-null object
strBIADecisionType        327849 non-null object
strCaseType               335603 non-null object
strLang                   334885 non-null object
strNat                    335446 non-null object
strProceedingIHP          167810 non-null object
strCustody                284495 non-null object
strProbono                583 non-null object
datAppealFiled_dt         335552 non-null datetime64[ns]
datBI

# Dedup for unique (idncase, idnproceeding)

~26% of remaining proceedings have multiple appeals 

In [113]:
appeals_per_proceeding = appeals_in_scope.groupby(['idnProceeding'])['idnAppeal'].nunique()
appeals_per_proceeding.value_counts(normalize=True)

1     0.737199
2     0.196977
3     0.047014
4     0.013153
5     0.003695
6     0.001233
7     0.000412
8     0.000121
9     0.000061
15    0.000044
10    0.000024
13    0.000020
11    0.000016
12    0.000008
20    0.000008
21    0.000004
14    0.000004
23    0.000004
Name: idnAppeal, dtype: float64

Most multiple appeals seem to involve 'MTR' after initial 'Case Appeal'. Should we use the last instance of 'MTR' as the final decision?

In [114]:
# examples of proceedings with multiple appeals 
sample_index = random.sample(appeals_per_proceeding[appeals_per_proceeding > 1].index, 1)
tblAppeal_df[tblAppeal_df['idnProceeding'].isin(sample_index)].T 

,282963,352738,434215,566596,741015
idnAppeal,4477066,4555446,4643663,4794537,5011887
idncase,3900854,3900854,3900854,3900854,3900854
idnProceeding,2302570,2302570,2302570,2302570,2302570
strAppealCategory,IJ,IJ,IJ,IJ,IJ
strAppealType,Case Appeal,MTR BIA,MTR BIA,MTR BIA,MTR BIA
datAppealFiled,2000-12-11 00:00:00,2002-11-29 00:00:00,2004-07-20 00:00:00,2007-07-26 00:00:00,2013-03-18 00:00:00
strFiledBy,A,A,A,A,A
datAttorneyE27,NaN,NaN,NaN,2007-07-26 00:00:00,2013-03-18 00:00:00
datBIADecision,2002-10-31 00:00:00,2003-01-21 00:00:00,2004-08-30 00:00:00,2008-01-02 00:00:00,2013-04-15 00:00:00
strBIADecision,SAF,DEN,DEN,DEN,TER


In [115]:
# check original proceedings for a given case 
master_dchen_df[master_dchen_df['idncase'] == 2241734].T

,4812094
idncase,2241734
nat,EG
case_type,DEP
c_asy_type,E
idnproceeding,361062
base_city_code,NEW
hearing_loc_code,NEW
dec_type,O
dec_code,V
other_comp,


~2% of cases have multiple proceedings 

In [116]:
proceedings_per_case = appeals_in_scope.groupby(['idncase'])['idnProceeding'].nunique()
proceedings_per_case.value_counts(normalize=True)

1    0.980875
2    0.018581
3    0.000540
4    0.000004
Name: idnProceeding, dtype: float64

In [117]:
# examples of cases with multiple proceedings 
sample_index = random.sample(proceedings_per_case[proceedings_per_case > 1].index, 1)
appeals_in_scope[appeals_in_scope['idncase'].isin(sample_index)].sort_values(by='datAppealFiled_dt').T 

,2862,447625,508599
idnAppeal,2952,4673411,4743635
idncase,2827170,2827170,2827170
idnProceeding,1121321,1121319,1121319
strAppealCategory,IJ,IJ,IJ
strAppealType,Case Appeal,Case Appeal,MTR BIA
datAppealFiled,1994-05-12 00:00:00,2005-02-17 00:00:00,2006-06-13 00:00:00
strFiledBy,A,A,A
datAttorneyE27,1994-05-12 00:00:00,2005-02-17 00:00:00,2006-06-13 00:00:00
datBIADecision,2000-11-09 00:00:00,2005-12-13 00:00:00,2006-12-19 00:00:00
strBIADecision,REM,DIS,DEN


In [118]:
# check original proceedings for a given case 
master_df[master_df['idncase'] == 5612569].T

,4112806,4112807,4112808,4112809,4112810
idncase,5.61257e+06,5.61257e+06,5.61257e+06,5.61257e+06,5.61257e+06
nat,ZA,ZA,ZA,ZA,ZA
case_type,RMV,RMV,RMV,RMV,RMV
c_asy_type,NaN,NaN,NaN,NaN,NaN
idnproceeding,4054290,4066075,4093297,4370491,5116381
base_city_code,FLO,PHO,ATL,ATL,ATL
hearing_loc_code,FLO,PHO,ATL,ATL,ATL
dec_type,NaN,NaN,O,NaN,O
dec_code,NaN,NaN,X,NaN,T
other_comp,T,C,NaN,O,NaN


** For each unique (case,proceeding) pairs with more than 1 appeal, pick the last appeal and dedup the rest ** 

In [119]:
appeals_deduped = appeals_in_scope.sort_values(by=['idncase', 'idnProceeding', 'datBIADecision_dt'], 
                                               ascending=[True, True, False])
appeals_deduped.drop_duplicates(subset=['idncase', 'idnProceeding'], keep='first', inplace=True)
appeals_deduped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247328 entries, 433199 to 731825
Data columns (total 29 columns):
idnAppeal                 247328 non-null int64
idncase                   247328 non-null int64
idnProceeding             247328 non-null int64
strAppealCategory         247328 non-null object
strAppealType             247328 non-null object
datAppealFiled            247280 non-null object
strFiledBy                247323 non-null object
datAttorneyE27            174280 non-null object
datBIADecision            247328 non-null object
strBIADecision            247328 non-null object
strBIADecisionType        241125 non-null object
strCaseType               247326 non-null object
strLang                   246996 non-null object
strNat                    247207 non-null object
strProceedingIHP          121173 non-null object
strCustody                206427 non-null object
strProbono                486 non-null object
datAppealFiled_dt         247280 non-null datetime64[ns]


**32% of appeals were granted**

In [121]:
# check % granted in final dataset 
appeals_deduped['granted'].value_counts(normalize=True)

0    0.679676
1    0.320324
Name: granted, dtype: float64

# Merge Datasets

#### Add variables from master-chen

In [122]:
# variables in processed data by Prof. Chen
print(master_dchen.columns.tolist())

['idncase', 'nat', 'case_type', 'c_asy_type', 'idnproceeding', 'base_city_code', 'hearing_loc_code', 'dec_type', 'dec_code', 'other_comp', 'attorney_flag', 'ij_code', 'tracid', 'comp_year', 'comp_month', 'comp_day', 'comp_date', 'osc_year', 'osc_month', 'osc_day', 'osc_date', 'input_year', 'input_month', 'input_day', 'input_date', 'lawyer', 'defensive', 'affirmative', 'case_type_string', '_mcase', 'dec_type_string', '_mdectype', 'outcome_recorded_in_field', 'dec_string', '_mdecproceeddec', '_mdecproceedoth', 'nat_string', '_mnat', 'base_city_street', 'base_city_string', 'base_city_state', 'base_city_zip5', 'base_city_zip4', 'base_city_phone', '_mbasecity', 'hearing_loc_string1', 'hearing_loc_string2', 'hearing_loc_street', 'hearing_loc_city', 'hearing_loc_state', 'hearing_loc_zip5', 'hearing_loc_phone', '_mhearingloc', 'judge_name_caps', '_mlookupijcode', 'cityid', 'judgeid', 'natid', 'venue_change', 'deport', 'relief_granted', 'remove', 'terminated', 'voluntary_departure', 'oral', 'wr

In [123]:
master_dchen.groupby(['grantordeny_chicago']).size()

grantordeny_chicago
1.0    609491
dtype: int64

In [124]:
master_dchen.isnull().sum()

idncase                                 0
nat                                     0
case_type                               0
c_asy_type                              0
idnproceeding                           0
base_city_code                          0
hearing_loc_code                        0
dec_type                                0
dec_code                                0
other_comp                              0
attorney_flag                     2907462
ij_code                                 0
tracid                             248615
comp_year                               0
comp_month                              0
comp_day                                0
comp_date                               0
osc_year                            32420
osc_month                           32420
osc_day                             32420
osc_date                            32420
input_year                          27549
input_month                         27549
input_day                         

In [125]:
master_dchen.sample(4).T

,3860857,2953703,2268814,4046051
idncase,5847870,4603040,4038746,6084714
nat,MX,CO,MX,MX
case_type,RMV,RMV,RMV,RMV
c_asy_type,,E,,
idnproceeding,4354405,5804791,2491047,4652789
base_city_code,SND,MIA,HOD,DEN
hearing_loc_code,CCA,MIA,HOD,WSI
dec_type,O,W,O,O
dec_code,X,T,X,X
other_comp,,,,


In [126]:
dchen_selected_features = ['idncase', 'idnproceeding', 'nat_string', 'ij_code', 'attorney_flag',
                           'lawyer', 'defensive', 'affirmative', 
                           'oral', 'written', 'base_city_code', 'base_city_state', 
                           'hearing_loc_city', 'hearing_loc_code', 'hearing_loc_state', 'venue_change', 
                           'comp_year', 'comp_month', 'comp_day', 
                           'osc_year', 'osc_month', 'osc_day', 'input_year', 'input_month', 'input_day',
                           'adj_time_start', 'flag_earlystarttime', 
                           'flag_mismatch_base_city', 'flag_mismatch_hearing', 'flag_datemismatch',
                           'case_type_string', 'dec_type_string', 'dec_string', 
                           'deport', 'relief_granted', 'remove', 'terminated', 'voluntary_departure', 
                           'deport_form', 'voluntary_form', 'deportation_proceeding', 
                           'exclusion_proceeding', 'removal_proceeding', 'asylum_only_proceeding', 
                           'withholding_only_proceeding'] 

In [127]:
# merge with master proceedings 
appeals_with_master = appeals_deduped.merge(master_dchen[dchen_selected_features], how='left', 
                                            left_on=['idncase', 'idnProceeding'], right_on=['idncase', 'idnproceeding'])\
                                     .drop(['idnproceeding_y', 'case_type_string_y'], axis=1)\
                                     .rename(columns={'idnproceeding_x': 'idnproceeding', 
                                                      'case_type_string_x': 'case_type_string'})
appeals_with_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247328 entries, 0 to 247327
Data columns (total 71 columns):
idnAppeal                      247328 non-null int64
idncase                        247328 non-null int64
idnProceeding                  247328 non-null int64
strAppealCategory              247328 non-null object
strAppealType                  247328 non-null object
datAppealFiled                 247280 non-null object
strFiledBy                     247323 non-null object
datAttorneyE27                 174280 non-null object
datBIADecision                 247328 non-null object
strBIADecision                 247328 non-null object
strBIADecisionType             241125 non-null object
strCaseType                    247326 non-null object
strLang                        246996 non-null object
strNat                         247207 non-null object
strProceedingIHP               121173 non-null object
strCustody                     206427 non-null object
strProbono                  

In [128]:
appeals_with_master.groupby(['case_type_string']).size()

case_type_string
                         2
ASYLUM ONLY CASE      3612
DEPORTATION          62343
EXCLUSION            16913
REMOVAL             163757
WITHHOLDING ONLY       701
dtype: int64

#### Output dataset for model training

In [129]:
# define mandatory features, without which data will be dropped 
mandatory_features = ['idnAppeal', 'granted', 'datAppealFiled_year', 'case_type_string'] 
appeals_with_master.dropna(subset=mandatory_features, inplace=True)

In [133]:
# save data with all features 
appeals_with_master.to_csv('data_all_features_2018-05-03.csv')

In [134]:
# define features to output for training 
output_features = ['idnAppeal', 'nat_string', 'strCustody', 'strProbono', 'case_type_string',
                   'ij_code', 'lawyer', 'defensive', 'affirmative', 'oral', 'written', 
                   'comp_year', 'osc_year', 'input_year', 'datAppealFiled_year', 'granted']
appeals_final = appeals_with_master[output_features].copy()

# generate output 
appeals_final.to_csv('data_for_model_2018-05-03.csv', index=False)